In [ ]:
# This is only a very early prototype of the interactive widget using three.js

import bpy
from pathlib import Path
import molecularnodes as mn

# Ensure the glTF exporter is activated
bpy.ops.preferences.addon_enable(module="io_scene_gltf2")

# Delete all mesh objects from the scene
bpy.ops.object.select_all(action="DESELECT")
bpy.ops.object.select_by_type(type="MESH")
bpy.ops.object.delete()

obj = mn.load.molecule_rcsb("7TYG", starting_style="cartoon", center_molecule=True)
obj.scale = [10, 10, 10]

bpy.context.view_layer.objects.active = obj
obj.modifiers["MolecularNodes"].node_group.nodes["MN_color_attribute_random"].inputs["Lightness"].default_value = 0.5
obj.modifiers["MolecularNodes"].node_group.nodes["MN_color_attribute_random"].inputs["Saturation"].default_value = 1
obj.modifiers["MolecularNodes"].node_group.nodes["MN_color_attribute_random"].inputs["Seed"].default_value = 1

for modifier in obj.modifiers:
    bpy.ops.object.modifier_apply(modifier=modifier.name)
bpy.ops.geometry.color_attribute_render_set(name="Color")


# Exporting the model to .glb format for Three.js
# Specify the path where you want to save the exported model
gltf_path = Path() / "_MY_MODEL_DATA.gltf"

bpy.ops.geometry.color_attribute_render_set(name="Color")

# Select only the object
bpy.ops.object.select_all(action="DESELECT")
obj.select_set(True)

# Export the selected obj object to a .glb file
bpy.ops.export_scene.gltf(
    filepath=str(gltf_path), export_format="GLTF_EMBEDDED", use_selection=True
)

data = gltf_path.read_text()

gltf_path.unlink()

from anywidget_3dviewer import Viewer

b = Viewer(gltf_data=data)
b